<a href="https://colab.research.google.com/github/Danyxyz/ADS_Projektarbeit/blob/main/AFEProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=59d991829dae81b05625c0604feb96ef2997d7b9d862010ad963b7460c98e0c8
  Stored in directory: /root/.cache/pip/wheels/42/56/cc/4a8bf86613aafd5b7f1b310477667c1fca5c51c3ae4124a003
Successfully built sklearn


In [4]:
from pandas.core.dtypes.inference import Number
from pandas.core.common import random_state
from pandas import DataFrame
import pandas as pd

In [7]:
#read in the data, only use relevant coloumns
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Gebaude_Bauzonen.csv",on_bad_lines="warn", engine="python", delimiter=',',usecols=['GGDENR','CH_BEZ_D','GKODE','GKODN'])

#Formatting, putting in a DataFrame
df_gebaude = pd.DataFrame(data)
df_gebaude = df_gebaude.rename(columns={'CH_BEZ_D': 'Bauzone'})
df_gebaude["Index"] = ""
df_gebaude.head(5)


,GGDENR,GKODE,GKODN,Bauzone,Index
0,1,2679647.268,1237500.347,Mischzonen,
1,1,2680635.900,1236936.229,Mischzonen,
2,1,2679016.828,1235828.156,Mischzonen,
3,1,2679677.869,1237453.256,Mischzonen,
4,1,2679692.997,1237491.284,Mischzonen,


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
#Read and format local communities
gemeindeData =  pd.read_csv("/content/drive/MyDrive/Colab Notebooks/main_gemeinde_data.csv", engine="python", delimiter=';',usecols=['Index','Gemeinde_Nr','K1_Hoch','K2_Mittel','K3_Tief'])
df_gemeindeData = pd.DataFrame(gemeindeData)
print(df_gemeindeData.head())

   Index  Gemeinde_Nr  K1_Hoch  K2_Mittel  K3_Tief
0      0          1.0     59.0       35.0     19.0
1      1          2.0    699.0      411.0    226.0
2      2          3.0    148.0       87.0     48.0
3      3          4.0    126.0       74.0     41.0
4      4          5.0    163.0       96.0     53.0


In [ ]:
for i in range(0, len(df_gebaude)-1):
  bfs = df_gebaude.GGDENR.iloc[i]
  x = i 
  for i in range(0, len(gemeindeData)-1):
    if bfs == gemeindeData.Gemeinde_Nr.iloc[i]:
      df_gebaude.Index.iloc[x] = gemeindeData.Index.iloc[i]
      break


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [11]:
from sklearn.cluster import KMeans
import numpy as np
from numpy import savetxt

# Wohnzone und eingeschränkte bauzonen, weitere bauzonen, verkehrzonen innerhlab der Bauzonen 20%
# Mittlere, Arbeits und Mischzonen 40%
# Hohe Dichte: Tourismus und Freizeitzonen, Zenturmszonen und Zonen für öffentliche Nutzung: 40%

#Loops through the GWR-Dataset
#Filter GWR-Dataset to communitites

low_zone_NORTH = []
low_zone_EAST = []
middle_zone_NORTH = []
middle_zone_EAST = []
high_zone_NORTH = []
high_zone_EAST = []
qgis_coordinate_list = []

#for i in range len(GemeineAnzahl):
for i in range(905,906):
  df_subset = df_gebaude[df_gebaude.GGDENR == 2]
  k_iterator = i - 1
  print(i)

  #Stores filtered communitites to the according zone - Low / Middle / High to perform the right weighting - K in kmeans
  #for i in range len(df_subset):
  for i in range(0,len(df_subset)-1):
    if df_subset.Bauzone.iloc[i] == 'Wohnzonen' or df_subset.Bauzone.iloc[i] == 'eingeschränkte Bauzonen' or df_subset.Bauzone.iloc[i] == 'weitere Bauzonen' or df_subset.Bauzone.iloc[i] == 'Verkehrszonen innerhalb der Bauzonen':
      low_zone_NORTH.append(df_subset.GKODN.iloc[i])
      low_zone_EAST.append(df_subset.GKODE.iloc[i])
    else: 
      if df_subset.Bauzone.iloc[i] == 'Arbeitszonen' or df_subset.Bauzone.iloc[i] == 'Mischzonen':
        middle_zone_NORTH.append(df_subset.GKODN.iloc[i])
        middle_zone_EAST.append(df_subset.GKODE.iloc[i])
      else:  
        high_zone_NORTH.append(df_subset.GKODN.iloc[i])
        high_zone_EAST.append(df_subset.GKODE.iloc[i])

  #-------------------------------------- ------------------------------------Perform Kmeansn
  #--------------------------------------------------------------------------Run kmeans on first list 
  df_low_zone_NORTH = pd.DataFrame(low_zone_NORTH)
  df_low_zone_EAST = pd.DataFrame(low_zone_EAST)
  df_middle_zone_NORTH = pd.DataFrame(middle_zone_NORTH)
  df_middle_zone_EAST = pd.DataFrame(middle_zone_EAST)
  df_high_zone_NORTH = pd.DataFrame(high_zone_NORTH)
  df_high_zone_EAST = pd.DataFrame(high_zone_EAST)

  df_low = pd.merge(df_low_zone_NORTH, df_low_zone_EAST, left_index=True, right_index=True)
  np_array_low = df_low.to_numpy()

  df_middle = pd.merge(df_middle_zone_NORTH, df_middle_zone_EAST, left_index=True, right_index=True)
  np_array_middle = df_middle.to_numpy()

  df_high = pd.merge(df_high_zone_NORTH, df_high_zone_EAST, left_index=True, right_index=True)
  np_array_high = df_low.to_numpy()

  #Begin Calculations with low
  k = df_gemeindeData.K1_Hoch[k_iterator]
  print(k)
  k = k.astype(int)
  k = round(k*0.2)
  
  kmeans = KMeans(n_clusters=k, random_state=0).fit(np_array_low)
  #Set the labels
  labels = kmeans.labels_
  centroids = kmeans.cluster_centers_

  #Store centroids of x and y individually
  centroids_x = centroids[:,0]
  centroids_y = centroids[:,1]

  coordinate_list = []
  for i in range(len(centroids_x)):
    number = np.round(centroids[i],2)
    coordinate_list.append(number)

  qgis_coordinate_list.extend(coordinate_list)

  #----------------------------------------------------------------------------------------------------------Run kmeans on seconnd secomd 
  df_middle_zone_NORTH = pd.DataFrame(middle_zone_NORTH)
  df_middle_zone_EAST = pd.DataFrame(middle_zone_EAST)

  df_middle = pd.merge(df_middle_zone_NORTH, df_middle_zone_EAST, left_index=True, right_index=True)
  np_array = df_middle.to_numpy()
  k = df_gemeindeData.K1_Hoch[k_iterator]
  k = k.astype(int)
  k = round(k*0.4)
  kmeans = KMeans(n_clusters=k, random_state=0).fit(np_array)
  #Set the labels
  labels = kmeans.labels_
  centroids = kmeans.cluster_centers_

  #Store centroids of x and y individually
  centroids_x = centroids[:,0]
  centroids_y = centroids[:,1]

  coordinate_list = []
  for i in range(len(centroids_x)):
    number = np.round(centroids[i],2)
    coordinate_list.append(number)

  qgis_coordinate_list.extend(coordinate_list)

  #Append to list instead of save as or writetxt

  #-------------------------------------------------------------------------------Run kmeans on third list 
  df_high_zone_NORTH = pd.DataFrame(high_zone_NORTH)
  df_high_zone_EAST = pd.DataFrame(high_zone_EAST)

  df_high = pd.merge(df_high_zone_NORTH, df_high_zone_EAST, left_index=True, right_index=True)
  np_array = df_high.to_numpy()
  k = df_gemeindeData.K1_Hoch[k_iterator]
  k = k.astype(int)
  k = round(k*0.4)
  
  kmeans = KMeans(n_clusters=k, random_state=0).fit(np_array)
  #Set the labels
  labels = kmeans.labels_
  centroids = kmeans.cluster_centers_

  #Store centroids of x and y individually
  centroids_x = centroids[:,0]
  centroids_y = centroids[:,1]

  coordinate_list = []
  for i in range(len(centroids_x)):
    number = np.round(centroids[i],2)
    coordinate_list.append(number)

  qgis_coordinate_list.extend(coordinate_list)
  k_iterator = k_iterator + 1

savetxt('Ladestationen_Birsfelden.csv', qgis_coordinate_list, delimiter=',',fmt='%f')









905
665.0


In [ ]:
k = df_gemeindeData.K1_Hoch[i-1]
print(k)
